In [2]:
# # --- 01_EDA_Preprocessing.ipynb ---
# # ----------------------------------

# # 1. Import Libraries
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# import os # Ditambahkan untuk operasi folder

# print("✅ Libraries berhasil diimpor.")

# # 2. Load Dataset
# # JALUR FILE DIPERBAIKI: Mengakses data dari folder 'data/raw'
# file_path = '../data/raw/dataR2.csv' 
# try:
#     df = pd.read_csv(file_path)
#     print(f"✅ Dataset '{file_path}' berhasil dimuat.")
# except FileNotFoundError:
#     print(f"❌ ERROR: File tidak ditemukan di jalur: {file_path}")
#     print("Pastikan file 'dataR2.csv' ada di folder '../data/raw/'.")
#     exit()

# # 3. Data Cleaning & Feature Definition
# df = df.rename(columns={'Classification': 'Diagnosis'})

# # Definisikan Fitur (X) dan Target (y)
# X = df.drop('Diagnosis', axis=1)
# # Mengubah label: 1 (Sehat) -> 0, 2 (Kanker) -> 1
# y = df['Diagnosis'] - 1

# print("\n✅ Variabel X (Fitur) dan y (Target) berhasil didefinisikan.")

# # 4. Eksplorasi Data Awal (EDA) - (Tidak diubah)

# # 5. Pembagian Data (80% Training, 20% Testing)
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

# # 6. Scaling (Standardization)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Kembalikan ke DataFrame (untuk menyimpan nama kolom)
# X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
# X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
# # Pastikan y_train dan y_test adalah Series untuk konsistensi
# y_train = pd.Series(y_train, name='Diagnosis', index=X_train.index)
# y_test = pd.Series(y_test, name='Diagnosis', index=X_test.index)


# print("✅ Data telah berhasil di-scale (Standardization).")

# # --- BARU: Menyimpan Data ke Disk untuk Modularity ---
# output_dir = '../data/processed'
# os.makedirs(output_dir, exist_ok=True) # Buat folder jika belum ada

# # Simpan data yang sudah di-scaled dan di-split
# X_train_scaled.to_csv(f'{output_dir}/X_train_scaled.csv', index=False)
# X_test_scaled.to_csv(f'{output_dir}/X_test_scaled.csv', index=False)

# # Simpan y_train dan y_test
# pd.DataFrame(y_train).to_csv(f'{output_dir}/y_train.csv', index=False)
# pd.DataFrame(y_test).to_csv(f'{output_dir}/y_test.csv', index=False)

# print(f"\n✅ Data pre-processed berhasil disimpan di: {output_dir}")

In [4]:
# ==========================================
# JUDUL: 01_EDA_Preprocessing.ipynb
# ==========================================

# 1. Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

# Konfigurasi tampilan
sns.set(style="whitegrid")
print("✅ Libraries berhasil diimpor.")

# ------------------------------------------
# 2. Load Dataset
# ------------------------------------------
file_path = '../data/raw/dataR2.csv' 

try:
    df = pd.read_csv(file_path)
    print(f"✅ Dataset berhasil dimuat. Ukuran awal: {df.shape}")
except FileNotFoundError:
    print(f"❌ ERROR: File tidak ditemukan di jalur: {file_path}")
    exit()

# Rename kolom agar lebih mudah dipahami
df = df.rename(columns={'Classification': 'Diagnosis'})

# ------------------------------------------
# 3. REVISI PENTING: Filter Data (Sesuai Paper)
# ------------------------------------------
# Paper Patrício et al. (2018) mengeksklusi peserta dengan BMI > 40.
# Kita WAJIB melakukan ini agar hasil valid secara komparasi.

initial_count = df.shape[0]
df_filtered = df[df['BMI'] <= 40].copy() # Filter BMI <= 40
final_count = df_filtered.shape[0]
dropped_count = initial_count - final_count

print("-" * 30)
print(f"🔍 PENERAPAN KRITERIA EKSKLUSI (Paper Ref):")
print(f"   - Sampel Awal  : {initial_count}")
print(f"   - Dibuang      : {dropped_count} (BMI > 40)")
print(f"   - Sampel Valid : {final_count}")
print("-" * 30)

# ------------------------------------------
# 4. Definisi Fitur (X) dan Target (y)
# ------------------------------------------
X = df_filtered.drop('Diagnosis', axis=1)

# Encoding Target:
# Data asli: 1 = Sehat, 2 = Kanker
# Kita ubah ke format standar ML: 0 = Sehat, 1 = Kanker
y = df_filtered['Diagnosis'] - 1 

print("✅ Target berhasil di-encode (0: Sehat, 1: Kanker).")

# ------------------------------------------
# 5. Pembagian Data (Splitting)
# ------------------------------------------
# Menggunakan stratify=y untuk menjaga proporsi kelas kanker/sehat seimbang
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ Data Split Selesai:")
print(f"   - Training set : {X_train.shape[0]} baris")
print(f"   - Testing set  : {X_test.shape[0]} baris")

# ------------------------------------------
# 6. Scaling (Standardization)
# ------------------------------------------
# PENTING: Fit scaler HANYA pada training data untuk mencegah Data Leakage
scaler = StandardScaler()

# Fit dan transform pada Train
X_train_scaled_array = scaler.fit_transform(X_train)

# Transform pada Test (JANGAN di-fit ulang!)
X_test_scaled_array = scaler.transform(X_test)

# Kembalikan ke format DataFrame agar nama kolom tidak hilang (Penting untuk Feature Importance nanti)
X_train_scaled = pd.DataFrame(X_train_scaled_array, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled_array, columns=X_test.columns)

# Pastikan y_train dan y_test tetap memiliki index yang benar
y_train = pd.Series(y_train.values, name='Diagnosis')
y_test = pd.Series(y_test.values, name='Diagnosis')

print("✅ Scaling selesai (Mean & Std diambil dari data training).")

# ------------------------------------------
# 7. Simpan Data Bersih
# ------------------------------------------
output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)

X_train_scaled.to_csv(f'{output_dir}/X_train_scaled.csv', index=False)
X_test_scaled.to_csv(f'{output_dir}/X_test_scaled.csv', index=False)
y_train.to_csv(f'{output_dir}/y_train.csv', index=False)
y_test.to_csv(f'{output_dir}/y_test.csv', index=False)

print(f"\n💾 SUKSES: Data preprocessing tersimpan di folder '{output_dir}'")
print("   Siap lanjut ke notebook 02_Modeling_Tuning.ipynb")

✅ Libraries berhasil diimpor.
✅ Dataset berhasil dimuat. Ukuran awal: (116, 10)
------------------------------
🔍 PENERAPAN KRITERIA EKSKLUSI (Paper Ref):
   - Sampel Awal  : 116
   - Dibuang      : 0 (BMI > 40)
   - Sampel Valid : 116
------------------------------
✅ Target berhasil di-encode (0: Sehat, 1: Kanker).
✅ Data Split Selesai:
   - Training set : 92 baris
   - Testing set  : 24 baris
✅ Scaling selesai (Mean & Std diambil dari data training).

💾 SUKSES: Data preprocessing tersimpan di folder '../data/processed'
   Siap lanjut ke notebook 02_Modeling_Tuning.ipynb
